### Import Libraries

This cell imports necessary Python libraries:
- `pandas` as `pd`: For data manipulation and analysis.
- `os`: For interacting with the operating system, like listing directory contents.
- `re`: For regular expressions, used here for pattern matching in filenames.
- `unicodedata`: For handling Unicode characters, particularly useful for normalizing text with diacritics.

In [2]:

import pandas as pd
import os
import re
import unicodedata

### Extract Company Names and Years from Filenames

This cell processes the filenames in the specified `txt_converted` folder. It extracts the company name and publishing year from each filename, assuming a `_YYYY.txt` pattern. It then creates a Pandas DataFrame (`file_df`) containing these extracted details, which will be used for merging with metadata.

In [3]:
# Path to your txt_converted folder
folder_path = './txt_converted'
files = os.listdir(folder_path)

company_names = []
publishing_years = []
filenames = []

for filename in files:
    if not filename.endswith('.txt'):
        continue

    base_name = filename[:-4]
    match = re.search(r'(\d{4})$', base_name)

    if match:
        year = int(match.group(1))
        company_name = base_name[:match.start()].strip()

        if company_name.endswith('_') or company_name.endswith('-'):
            company_name = company_name[:-1].strip()

        company_names.append(company_name)
        publishing_years.append(year)
        filenames.append(filename)

file_df = pd.DataFrame({
    'filename': filenames,
    'company_name': company_names,
    'year': publishing_years
})

print(f"Total text files: {len(file_df)}")
print(file_df.head())

Total text files: 5790
                                            filename  \
0                                  Camposol_2015.txt   
1                                   Anasoft_2015.txt   
2                  YunnanCoal&EnergyCompany_2013.txt   
3  ChinaDevelopmentFinancialHoldingCorporation(CD...   
4                                     AECOM_2016.txt   

                                        company_name  year  
0                                           Camposol  2015  
1                                            Anasoft  2015  
2                           YunnanCoal&EnergyCompany  2013  
3  ChinaDevelopmentFinancialHoldingCorporation(CD...  2013  
4                                              AECOM  2016  


### Load Metadata and Initial Matching

This cell loads the `tailored_metadata.csv` file, which contains additional information about the reports. It then performs an initial merge between the `file_df` (created in the previous step) and the `metadata` DataFrame, using the `filename` column. This step identifies how many files from the `txt_converted` folder successfully match with entries in the metadata.

In [4]:
# Load metadata
metadata = pd.read_csv('./tailored_metadata.csv')
print(f"Metadata columns: {metadata.columns.tolist()}")

# Merge on filename
matched = pd.merge(
    file_df,
    metadata,
    left_on='filename',
    right_on='file_full_name',
    how='left'
)

matched_count = matched['Name'].notna().sum()
print(f"Matched: {matched_count} out of {len(file_df)}")
print(f"Match rate: {matched_count/len(file_df)*100:.1f}%")

Metadata columns: ['Name', 'Year', 'file', 'Organization_type', 'Size', 'Sector', 'Sec_SASB', 'Country', 'Region', 'OECD', 'english_non_english', 'file_full_name']
Matched: 5790 out of 5790
Match rate: 100.0%


### Identify Unmatched Files and Analyze Diacritics

After the initial merge, this cell identifies and inspects the files that did not find a match in the metadata. It specifically checks for the presence of diacritics (accents or special characters) in the filenames, as these can often cause mismatch issues. It prints a list of unmatched files and a count of those containing diacritics.

In [5]:
# After Block 3 merge, check what didn't match
unmatched = matched[matched['Name'].isna()].copy()
print(f"\n{'='*60}")
print(f"UNMATCHED FILES: {len(unmatched)}")
print(f"{'='*60}")

if len(unmatched) > 0:
    print("\nFirst 20 unmatched filenames:")
    for idx, row in unmatched.head(20).iterrows():
        print(f"  {row['filename']}")

    print("\nSample of unmatched company names extracted from filenames:")
    print(unmatched['company_name'].value_counts().head(15))

    # Check if these might be Unicode issues
    import unicodedata
    def has_diacritics(text):
        if pd.isna(text):
            return False
        text = str(text)
        normalized = unicodedata.normalize('NFD', text)
        for c in normalized:
            if unicodedata.combining(c):
                return True
        return False

    unmatched['has_diacritics'] = unmatched['filename'].apply(has_diacritics)
    diacritic_count = unmatched['has_diacritics'].sum()
    print(f"\nFiles with diacritics/accents: {diacritic_count}")

    if diacritic_count > 0:
        print("\nExamples of files with diacritics:")
        print(unmatched[unmatched['has_diacritics']]['filename'].head(10).tolist())
else:
    print("✓ All files matched successfully!")


UNMATCHED FILES: 0
✓ All files matched successfully!


### Re-match Files Using Unicode Normalization

This cell attempts to resolve mismatches caused by Unicode characters. It normalizes both the `filename` and `file_full_name` columns to a common format (removing diacritics) and then performs a re-merge. This step aims to increase the match rate by standardizing character representations. It then updates the `matched` DataFrame with the newly found matches.

In [6]:
print(f"\n{'='*60}")
print(f"ADDING NEWLY MATCHED FILES")
print(f"{'='*60}")

if 'missing_found' in locals() and len(missing_found) > 0:
    print(f"\nFound {len(missing_found)} files via Unicode normalization:")
    print("-" * 40)

    # Show the files being added
    for i, f in enumerate(missing_found[:20]):  # Show first 20
        print(f"  ✅ Adding: {f}")
    if len(missing_found) > 20:
        print(f"  ... and {len(missing_found) - 20} more")

    # Get metadata for found files
    found_metadata = metadata[metadata['file_full_name'].isin(missing_found)].copy()
    found_metadata['filename'] = found_metadata['file_full_name']

    # Get file_df entries
    found_files = file_df[file_df['filename'].isin(missing_found)].copy()

    # Merge
    found_merged = pd.merge(
        found_files,
        found_metadata,
        on='filename',
        how='left'
    )

    # Separate already matched
    already_matched = matched[matched['Name'].notna()].copy()

    # Store counts before
    before_count = len(already_matched)

    # Combine
    matched = pd.concat([already_matched, found_merged], ignore_index=True)

    # Remove any duplicates
    if matched.duplicated(subset=['filename']).any():
        dup_count = matched.duplicated(subset=['filename']).sum()
        print(f"\n⚠️ Found {dup_count} duplicates, removing...")
        matched = matched.drop_duplicates(subset=['filename'])

    print(f"\n{'='*40}")
    print(f"RESULTS:")
    print(f"{'='*40}")
    print(f"Previously matched: {before_count}")
    print(f"Newly added: {len(found_merged)}")
    print(f"Total matched now: {len(matched)}")

    # Verify we have all files
    all_files_count = len(file_df)
    matched_files_count = matched['filename'].nunique()

    if matched_files_count == all_files_count:
        print(f"\n✅ SUCCESS: All {all_files_count} files are now matched!")
    else:
        print(f"\n⚠️ Still missing: {all_files_count - matched_files_count} files")

    # Show if any still missing metadata
    still_no_metadata = matched[matched['Name'].isna()]
    if len(still_no_metadata) > 0:
        print(f"\nFiles without metadata (will be removed later): {len(still_no_metadata)}")
        print("First few:")
        for idx, row in still_no_metadata.head(5).iterrows():
            print(f"  ❌ {row['filename']}")
else:
    print("No files were found in Unicode matching")

print(f"\n{'='*60}")


ADDING NEWLY MATCHED FILES
No files were found in Unicode matching



### Check for and Remove Empty Files

This cell verifies the integrity of the text files by checking if any of the matched files are truly empty (0 bytes in size). Empty files are identified and then removed from the dataset, as they would not contain any useful information for analysis. It also provides a breakdown of empty files by sector and country to identify any patterns.

In [7]:
print(f"\n{'='*60}")
print(f"CHECKING FOR EMPTY FILES (0 BYTES)")
print(f"{'='*60}")

folder_path = './txt_converted'

def is_truly_empty(filename):
    filepath = os.path.join(folder_path, filename)
    try:
        return os.path.getsize(filepath) == 0
    except Exception as e:
        print(f"Error checking {filename}: {e}")
        return True

# Check all matched files
matched['file_empty'] = matched['filename'].apply(is_truly_empty)
empty_count = matched['file_empty'].sum()
print(f"\nTotal files checked: {len(matched)}")
print(f"Truly empty files (0 bytes): {empty_count}")

# Show empty files if any
if empty_count > 0:
    print(f"\n{'='*40}")
    print(f"EMPTY FILES TO BE REMOVED:")
    print(f"{'='*40}")
    empty_files = matched[matched['file_empty']].copy()

    # Show them grouped by possible reasons
    print("\nFirst 20 empty files:")
    for idx, row in empty_files.head(20).iterrows():
        print(f"  ❌ {row['filename']} - {row.get('Name', 'Unknown')}")

    if empty_count > 20:
        print(f"  ... and {empty_count - 20} more")

    # Check if certain sectors/countries have more empty files
    if 'Sector' in empty_files.columns:
        print(f"\nEmpty files by sector:")
        print(empty_files['Sector'].value_counts().head(10))

    if 'Country' in empty_files.columns:
        print(f"\nEmpty files by country:")
        print(empty_files['Country'].value_counts().head(10))

    # Remove empty files
    final_data = matched[~matched['file_empty']].copy()
    print(f"\n{'='*40}")
    print(f"AFTER REMOVING EMPTY FILES:")
    print(f"{'='*40}")
    print(f"Files kept: {len(final_data)}")
    print(f"Files removed: {empty_count}")

else:
    print(f"\n✅ No empty files found!")
    final_data = matched.copy()

print(f"\nFinal dataset size: {len(final_data)} reports")


CHECKING FOR EMPTY FILES (0 BYTES)

Total files checked: 5790
Truly empty files (0 bytes): 108

EMPTY FILES TO BE REMOVED:

First 20 empty files:
  ❌ DaiwaHouse_2008.txt - Daiwa House
  ❌ OceanwideRealEstateGroup_2015.txt - Oceanwide Real Estate Group
  ❌ DaiwaHouse_2007.txt - Daiwa House
  ❌ ChinaHuanengGroup_2017.txt - China Huaneng Group
  ❌ FormosaSumcoTechnologyCorporation(FST)_2014.txt - Formosa Sumco Technology Corporation (FST)
  ❌ LojasAmericanas_2017.txt - Lojas Americanas
  ❌ TimeWarnerCable_2015.txt - Time Warner Cable
  ❌ KaohsiungMunicipalSiaogangHospital_2015.txt - Kaohsiung Municipal  Siaogang Hospital
  ❌ L&THydrocarbonEngineering_2017.txt - L&T Hydrocarbon Engineering
  ❌ FundaciónSaldarriagaConcha(SaldarriagaConchaFoundation)_2016.txt - Fundación Saldarriaga Concha (Saldarriaga Concha Foundation)
  ❌ JandeNulGroup_2016.txt - Jan de Nul Group
  ❌ BaosteelGroupCorporation_2014.txt - Baosteel Group Corporation
  ❌ SporveienOslo_2017.txt - Sporveien Oslo
  ❌ UnimedCuiab

### Save Final Dataset and Summary

This cell saves the cleaned and filtered dataset (`final_data`) to a new CSV file named `final_reports_clean.csv` in your Google Drive. It then prints a summary of the final dataset, including the total number of reports, unique companies, sectors, countries, and the distribution of English vs. non-English reports.

In [8]:
print(f"\n{'='*60}")
print(f"SAVING FINAL DATASET")
print(f"{'='*60}")

final_data.to_csv('./final_reports_clean.csv', index=False)


# Final summary
print(f"\n{'='*60}")
print(f"FINAL DATASET SUMMARY")
print(f"{'='*60}")
print(f"Total reports: {len(final_data)}")
print(f"Unique companies: {final_data['Name'].nunique()}")
print(f"Unique sectors: {final_data['Sector'].nunique()}")
print(f"Unique countries: {final_data['Country'].nunique()}")
print(f"\nEnglish reports: {final_data['english_non_english'].value_counts().get('english', 0)}")
print(f"Non-English reports: {final_data['english_non_english'].value_counts().get('non-english', 0)}")


SAVING FINAL DATASET

FINAL DATASET SUMMARY
Total reports: 5682
Unique companies: 2972
Unique sectors: 38
Unique countries: 91

English reports: 1653
Non-English reports: 4029


### List All Sectors and Countries

This cell generates and displays a complete list of all unique sectors and countries present in the `final_data` DataFrame, along with the count of reports for each. It also saves these lists to separate CSV files (`all_sectors_list.csv` and `all_countries_list.csv`) for easy access and review.

In [9]:
print(f"\n{'='*60}")
print(f"ALL SECTORS - COMPLETE LIST")
print(f"{'='*60}")

# Get all sectors with counts
all_sectors = final_data['Sector'].value_counts().sort_values(ascending=False)

# Print each sector with count
for sector, count in all_sectors.items():
    print(f"{sector}: {count}")

# Save to file for easy viewing


print(f"\n{'='*60}")
print(f"ALL COUNTRIES - COMPLETE LIST")
print(f"{'='*60}")

# Get all countries with counts
all_countries = final_data['Country'].value_counts().sort_values(ascending=False)

# Print each country with count
for country, count in all_countries.items():
    print(f"{country}: {count}")



print(f"\n{'='*60}")
print(f"SUMMARY")
print(f"{'='*60}")
print(f"Total sectors: {len(all_sectors)}")
print(f"Total countries: {len(all_countries)}")
print(f"Total reports: {len(final_data)}")


ALL SECTORS - COMPLETE LIST
Financial Services: 606
Other: 567
Food and Beverage Products: 364
Chemicals: 324
Energy: 312
Equipment: 289
Technology Hardware: 273
Real Estate: 203
Metals Products: 186
Construction: 183
Energy Utilities: 165
Healthcare Products: 164
Automotive: 158
Telecommunications: 156
Conglomerates: 153
Retailers: 142
Construction Materials: 127
Mining: 121
Logistics: 121
Non-Profit / Services: 111
Commercial Services: 98
Tourism/Leisure: 76
Consumer Durables: 74
Computers: 73
Textiles and Apparel: 67
Aviation: 67
Household and Personal Products: 66
Water Utilities: 59
Healthcare Services: 56
Forest and Paper Products: 53
Railroad: 51
Agriculture: 49
Public Agency: 48
Media: 48
Universities: 34
Waste Management: 24
Toys: 10
Tobacco: 4

ALL COUNTRIES - COMPLETE LIST
Mainland China: 1407
Japan: 1097
Taiwan: 670
United States of America: 320
Germany: 131
India: 109
Colombia: 101
Spain: 92
Hong Kong: 90
Canada: 88
Finland: 86
Brazil: 80
Greece: 80
Russian Federation: 80

                                               filename  \
0                                     Camposol_2015.txt   
1                                      Anasoft_2015.txt   
2                     YunnanCoal&EnergyCompany_2013.txt   
3     ChinaDevelopmentFinancialHoldingCorporation(CD...   
4                                        AECOM_2016.txt   
...                                                 ...   
5785  TianjinSamsungTechwinOpto-ElectronicCompany_20...   
5786             ControladoraComercialMexicana_2012.txt   
5787                        SinomachAutomobile_2014.txt   
5788                                     SNPTC_2016.txt   
5789                              FujiElectric_2015.txt   

                                           company_name  year  \
0                                              Camposol  2015   
1                                               Anasoft  2015   
2                              YunnanCoal&EnergyCompany  2013   
3     ChinaDevelopmentFinancial

### Analyze Sectors and Countries by Language

This cell performs a cross-tabulation to show the distribution of reports by `Sector` and `Country` against their `english_non_english` status. It provides insights into which sectors and countries have more English or non-English reports, displaying the top 20 entries for each category.

In [10]:
print(f"\n{'='*60}")
print(f"SECTORS BY LANGUAGE (Top 20)")
print(f"{'='*60}")

# Cross-tab of sectors by language
sector_lang = pd.crosstab(final_data['Sector'], final_data['english_non_english'])
print(sector_lang.head(20))

print(f"\n{'='*60}")
print(f"COUNTRIES BY LANGUAGE (Top 20)")
print(f"{'='*60}")

# Cross-tab of countries by language
country_lang = pd.crosstab(final_data['Country'], final_data['english_non_english'])
print(country_lang.head(20))


SECTORS BY LANGUAGE (Top 20)
english_non_english              english  non-english
Sector                                               
Agriculture                           13           36
Automotive                            49          109
Aviation                              24           43
Chemicals                             64          260
Commercial Services                   47           51
Computers                             23           50
Conglomerates                         47          106
Construction                          47          136
Construction Materials                30           97
Consumer Durables                     28           46
Energy                               103          209
Energy Utilities                      53          112
Equipment                             79          210
Financial Services                   184          422
Food and Beverage Products            92          272
Forest and Paper Products             28           2

In [11]:
print(f"\n{'='*60}")
print(f"TOP 5 SECTORS - REPORTS PER YEAR")
print(f"{'='*60}")

# Get top 5 sectors
top_sectors = final_data['Sector'].value_counts().head(5).index.tolist()

# Filter to top sectors
top_sectors_data = final_data[final_data['Sector'].isin(top_sectors)]

# Pivot table: years vs sectors
year_sector = pd.crosstab(top_sectors_data['Year'], top_sectors_data['Sector'])
print(year_sector)


TOP 5 SECTORS - REPORTS PER YEAR
Sector  Chemicals  Energy  Financial Services  Food and Beverage Products  \
Year                                                                        
2000            0       0                   1                           0   
2002            0       0                   1                           0   
2003            0       1                   0                           1   
2004            0       0                   1                           2   
2005            1       0                   3                           1   
2006            6       4                   1                           3   
2007            9       7                   2                           5   
2008            9       9                  10                           7   
2009           10       6                  12                           6   
2010           12      14                  22                          12   
2011           10      18                 

In [12]:
# Find companies with reports in both periods
pre_years = [2005, 2006, 2007]
post_years = [2009, 2010, 2011]

pre_companies = set(final_data[final_data['Year'].isin(pre_years)]['Name'])
post_companies = set(final_data[final_data['Year'].isin(post_years)]['Name'])

survivors = pre_companies.intersection(post_companies)
print(f"Companies that reported before AND after crisis: {len(survivors)}")

for s in survivors:
    print(s)

Companies that reported before AND after crisis: 52
Taiwan Mobile (TWM)
Takasago
Karsten
TERUMO
Rinnai
Ono Pharmaceutical
Exeo
MAEDA
NSK Ltd.
Yulon Motor Co., Ltd.
Shikoku Electric Power
TOPCON
Kureha Chemical Industries
House Foods
Madison Gas and Electric
Taiheiyo Cement
UNITIKA
Iwatani
Oki Electric Industry
Egged Israel Transport Cooperative Society
Baoshan Iron & Steel
Comarca Gipuzkoa - Osakidetza
Banrisul
SWCC
SOMAR
China Datang Corporation (CDT)
Eurochem
Obayashi
Megmilk
KOBE STEEL, LTD. (Kobelco)
Procter & Gamble
Cisco Systems, Inc.
tok
Okamura
ESPEC
Konica Minolta Group
Showa Sangyo
Sawafuji
Joshin
Kyodo Printing Group
Canfor Corp.
Itoham
Sumitomo Riko
Lintec
Zeon Corp.
Penta-Ocean
TOA OIL
Osaka Gas
J-Power
Mandom
Daiwa House
Tsumura & Co.


In [13]:
# Check English vs Non-English for your target sectors
sectors_of_interest = ['Financial Services', 'Energy', 'Mining', 'Utilities', 'Construction',
                       'Food and Beverage Products', 'Retailers']

# Filter to your sectors
target_data = final_data[final_data['Sector'].isin(sectors_of_interest)]

# English breakdown by sector
print("ENGLISH REPORTS BY SECTOR")
print("-" * 50)
eng_by_sector = target_data[target_data['english_non_english'] == 'english']['Sector'].value_counts()
print(eng_by_sector)

print("\nNON-ENGLISH REPORTS BY SECTOR")
print("-" * 50)
noneng_by_sector = target_data[target_data['english_non_english'] == 'non-english']['Sector'].value_counts()
print(noneng_by_sector)

# Total English in your target sectors
total_eng = target_data[target_data['english_non_english'] == 'english'].shape[0]
print(f"\nTOTAL ENGLISH REPORTS IN TARGET SECTORS: {total_eng}")

ENGLISH REPORTS BY SECTOR
--------------------------------------------------
Sector
Financial Services            184
Energy                        103
Food and Beverage Products     92
Construction                   47
Retailers                      45
Mining                         42
Name: count, dtype: int64

NON-ENGLISH REPORTS BY SECTOR
--------------------------------------------------
Sector
Financial Services            422
Food and Beverage Products    272
Energy                        209
Construction                  136
Retailers                      97
Mining                         79
Name: count, dtype: int64

TOTAL ENGLISH REPORTS IN TARGET SECTORS: 513


In [14]:
# See all English counts
eng_counts = final_data[final_data['english_non_english'] == 'english']['Sector'].value_counts()
print(eng_counts.head(20))


Sector
Financial Services                 184
Other                              122
Energy                             103
Food and Beverage Products          92
Equipment                           79
Technology Hardware                 75
Real Estate                         71
Chemicals                           64
Energy Utilities                    53
Healthcare Products                 52
Automotive                          49
Conglomerates                       47
Construction                        47
Commercial Services                 47
Telecommunications                  46
Retailers                           45
Logistics                           45
Mining                              42
Metals Products                     42
Household and Personal Products     36
Name: count, dtype: int64


In [15]:
# Define your three groups
industrial = ['Energy', 'Energy Utilities', 'Mining', 'Metals Products',
              'Chemicals', 'Construction', 'Automotive', 'Equipment']

financial = ['Financial Services', 'Real Estate', 'Commercial Services',
             'Logistics', 'Conglomerates']

consumer = ['Food and Beverage Products', 'Retailers', 'Technology Hardware',
            'Telecommunications', 'Healthcare Products', 'Household and Personal Products']




# Create group labels
final_data['Group'] = 'Other'  # default
final_data.loc[final_data['Sector'].isin(industrial), 'Group'] = 'Industrial'
final_data.loc[final_data['Sector'].isin(financial), 'Group'] = 'Financial'
final_data.loc[final_data['Sector'].isin(consumer), 'Group'] = 'Consumer'

# Filter to only these groups and English
analysis_data = final_data[
    (final_data['Group'] != 'Other') &
    (final_data['english_non_english'] == 'english')
].copy()

print(f"Final analysis dataset: {len(analysis_data)} English reports")
print(analysis_data['Group'].value_counts())

Final analysis dataset: 1219 English reports
Group
Industrial    479
Financial     394
Consumer      346
Name: count, dtype: int64


In [16]:
print(f"Total English reports: {len(analysis_data)}")
print(f"Year range: {analysis_data['Year'].min()} - {analysis_data['Year'].max()}")

Total English reports: 1219
Year range: 2002 - 2018


In [17]:
# Filter to Era 1: 2000-2008
era1 = analysis_data[analysis_data['Year'].between(2000, 2008)].copy()

print("ERA 1 (2000-2008)")
print("=" * 50)
print(f"Total English reports: {len(era1)}")
print("\nBy group:")
print(era1['Group'].value_counts())
print("\nBy year:")
print(era1['Year'].value_counts().sort_index())

ERA 1 (2000-2008)
Total English reports: 56

By group:
Group
Consumer      25
Industrial    21
Financial     10
Name: count, dtype: int64

By year:
Year
2002     3
2003     1
2004     1
2005     4
2006     6
2007    21
2008    20
Name: count, dtype: int64


In [18]:
# Check non-English in 2009-2012
non_eng_gap = final_data[
    (final_data['english_non_english'] == 'non-english') &
    (final_data['Year'].between(2009, 2012))
].copy()

print(f"Non-English reports 2009-2012: {len(non_eng_gap)}")

Non-English reports 2009-2012: 577


In [19]:
# For Industrial group
industrial_data = analysis_data[analysis_data['Group'] == 'Industrial']
print("INDUSTRIAL GROUP - Reports by Country")
print("=" * 60)
print(industrial_data['Country'].value_counts().head(20))
print("\n")

# For Financial group
financial_data = analysis_data[analysis_data['Group'] == 'Financial']
print("FINANCIAL GROUP - Reports by Country")
print("=" * 60)
print(financial_data['Country'].value_counts().head(20))
print("\n")

# For Consumer group
consumer_data = analysis_data[analysis_data['Group'] == 'Consumer']
print("CONSUMER GROUP - Reports by Country")
print("=" * 60)
print(consumer_data['Country'].value_counts().head(20))

INDUSTRIAL GROUP - Reports by Country
Country
Japan                                                   142
United States of America                                 68
India                                                    43
Canada                                                   19
Germany                                                  17
United Kingdom of Great Britain and Northern Ireland     17
France                                                   17
Finland                                                  15
Korea, Republic of                                       12
Switzerland                                              11
South Africa                                             10
Russian Federation                                        9
Italy                                                     9
Australia                                                 8
Poland                                                    8
Netherlands                                           

In [20]:
# See companies with their countries and report counts
company_summary = analysis_data.groupby(['Name', 'Country']).size().reset_index(name='report_count')
company_summary = company_summary.sort_values('report_count', ascending=False)

print("TOP 50 COMPANIES - WITH COUNTRY AND REPORT COUNT")
print("=" * 80)
print(company_summary.head(50).to_string(index=False))

TOP 50 COMPANIES - WITH COUNTRY AND REPORT COUNT
                         Name                                              Country  report_count
                       TERUMO                                                Japan             9
              Infosys Limited                                                India             8
            Jtekt Corporation                                                Japan             7
                      Iwatani                                                Japan             7
                     NSK Ltd.                                                Japan             7
                   Iino Lines                                                Japan             7
           Shin-Etsu Chemical                                                Japan             6
                       Sponda                                              Finland             6
                       Foster                                                J

In [21]:
# See the 78 English reports from Japan 2009-2012
japan_english_gap = final_data[
    (final_data['Country'] == 'Japan') &
    (final_data['Year'].between(2009, 2012)) &
    (final_data['english_non_english'] == 'english')
].copy()

print(f"Japanese English reports 2009-2012: {len(japan_english_gap)}")
print("\nBy year:")
print(japan_english_gap['Year'].value_counts().sort_index())
print("\nSample companies:")
print(japan_english_gap['Name'].head(10).tolist())

Japanese English reports 2009-2012: 78

By year:
Year
2009     6
2010    12
2011    26
2012    34
Name: count, dtype: int64

Sample companies:
['Iino Lines', 'Calsonic Kansei', "O'Hara Inc.", 'Chubu Electric Power', 'TERUMO', 'Foster', 'Brother', 'Sega Sammy Holdings', 'Jtekt Corporation', 'Iwatani']


In [22]:
analysis_data = final_data[
    (final_data['Group'] != 'Other') &
    (final_data['english_non_english'] == 'english')
].copy()
gap_check = analysis_data[analysis_data['Year'].between(2009, 2012)]
print(len(gap_check))

236


In [23]:
# Get the 78 Japanese English reports
japan_78 = final_data[
    (final_data['Country'] == 'Japan') &
    (final_data['Year'].between(2009, 2012)) &
    (final_data['english_non_english'] == 'english')
].copy()

print("Sector names in these 78 reports:")
print(japan_78['Sector'].value_counts())

Sector names in these 78 reports:
Sector
Equipment                 10
Energy                     6
Automotive                 6
Chemicals                  6
Financial Services         6
Metals Products            5
Technology Hardware        5
Retailers                  4
Healthcare Products        4
Energy Utilities           4
Logistics                  3
Construction               3
Other                      3
Railroad                   3
Conglomerates              2
Real Estate                2
Mining                     2
Telecommunications         1
Construction Materials     1
Computers                  1
Textiles and Apparel       1
Name: count, dtype: int64


In [24]:
# English reports only during 2009-2012
english_2009_2012 = final_data[
    (final_data['Year'].between(2009, 2012)) &
    (final_data['english_non_english'] == 'english')
].copy()

print(f"ENGLISH REPORTS 2009-2012: {len(english_2009_2012)}")
print("\nBY YEAR:")
print(english_2009_2012['Year'].value_counts().sort_index())
print("\nBY SECTOR (TOP 10):")
print(english_2009_2012['Sector'].value_counts().head(10))
print("\nBY COUNTRY (TOP 10):")
print(english_2009_2012['Country'].value_counts().head(10))
print("\nBY GROUP:")
if 'Group' in english_2009_2012.columns:
    print(english_2009_2012['Group'].value_counts())

ENGLISH REPORTS 2009-2012: 312

BY YEAR:
Year
2009     32
2010     55
2011    101
2012    124
Name: count, dtype: int64

BY SECTOR (TOP 10):
Sector
Financial Services            40
Other                         20
Equipment                     18
Energy                        18
Technology Hardware           14
Food and Beverage Products    14
Healthcare Products           12
Telecommunications            12
Automotive                    12
Chemicals                     11
Name: count, dtype: int64

BY COUNTRY (TOP 10):
Country
Japan                       78
United States of America    50
Canada                      18
Hong Kong                   13
South Africa                12
Australia                   11
India                       11
Netherlands                  9
Finland                      8
Mainland China               8
Name: count, dtype: int64

BY GROUP:
Group
Industrial    100
Other          76
Financial      68
Consumer       68
Name: count, dtype: int64


In [25]:
# First, make sure your group lists match EXACT sector names
industrial = ['Energy', 'Energy Utilities', 'Mining', 'Metals Products',
              'Chemicals', 'Construction', 'Automotive', 'Equipment']

financial = ['Financial Services', 'Real Estate', 'Commercial Services',
             'Logistics', 'Conglomerates']

consumer = ['Food and Beverage Products', 'Retailers', 'Technology Hardware',
            'Telecommunications', 'Healthcare Products', 'Household and Personal Products']

# Create group labels (overwriting any existing 'Group' column)
final_data['Group'] = 'Other'
final_data.loc[final_data['Sector'].isin(industrial), 'Group'] = 'Industrial'
final_data.loc[final_data['Sector'].isin(financial), 'Group'] = 'Financial'
final_data.loc[final_data['Sector'].isin(consumer), 'Group'] = 'Consumer'

# Now create analysis_data with ALL English reports
analysis_data = final_data[
    (final_data['english_non_english'] == 'english')
].copy()

print(f"Total English reports: {len(analysis_data)}")
print("\nBy group:")
print(analysis_data['Group'].value_counts())
print("\nBy year:")
print(analysis_data['Year'].value_counts().sort_index())

Total English reports: 1653

By group:
Group
Industrial    479
Other         434
Financial     394
Consumer      346
Name: count, dtype: int64

By year:
Year
2002      4
2003      2
2004      2
2005      6
2006      9
2007     24
2008     24
2009     32
2010     55
2011    101
2012    124
2013    195
2014    238
2015    245
2016    325
2017    264
2018      3
Name: count, dtype: int64


In [26]:
# Filter to 2013-2016, exclude Other group
analysis_2013_2016 = analysis_data[
    (analysis_data['Year'].between(2000, 2016)) &
    (analysis_data['Group'] != 'Other')
].copy()

print("ENGLISH REPORTS 2000-2016 (EXCLUDING OTHER)")
print("=" * 60)
print(f"Total: {len(analysis_2013_2016)}")
print("\nBy group:")
print(analysis_2013_2016['Group'].value_counts())
print("\nBy year:")
print(analysis_2013_2016['Year'].value_counts().sort_index())

ENGLISH REPORTS 2000-2016 (EXCLUDING OTHER)
Total: 1022

By group:
Group
Industrial    398
Financial     332
Consumer      292
Name: count, dtype: int64

By year:
Year
2002      3
2003      1
2004      1
2005      4
2006      6
2007     21
2008     20
2009     25
2010     44
2011     77
2012     90
2013    149
2014    173
2015    182
2016    226
Name: count, dtype: int64


In [27]:
# For each group, show sectors and countries

groups = ['Industrial', 'Financial', 'Consumer']

for group in groups:
    print(f"\n{'='*60}")
    print(f"{group} GROUP - SECTORS")
    print(f"{'='*60}")

    group_data = analysis_data[analysis_data['Group'] == group]

    # Sectors within this group
    print(group_data['Sector'].value_counts().head(10))

    print(f"\n{group} GROUP - COUNTRIES")
    print(f"{'='*60}")

    # Countries within this group
    print(group_data['Country'].value_counts().head(10))
    print("\n")


Industrial GROUP - SECTORS
Sector
Energy              103
Equipment            79
Chemicals            64
Energy Utilities     53
Automotive           49
Construction         47
Metals Products      42
Mining               42
Name: count, dtype: int64

Industrial GROUP - COUNTRIES
Country
Japan                                                   142
United States of America                                 68
India                                                    43
Canada                                                   19
Germany                                                  17
United Kingdom of Great Britain and Northern Ireland     17
France                                                   17
Finland                                                  15
Korea, Republic of                                       12
Switzerland                                              11
Name: count, dtype: int64



Financial GROUP - SECTORS
Sector
Financial Services     184
Real Estate         

In [28]:
# Create company summary table
company_summary = analysis_data.groupby(
    ['Name', 'Sector', 'Country', 'Group']
).size().reset_index(name='report_count')

# Sort by report count (highest first)
company_summary = company_summary.sort_values('report_count', ascending=False)

# Display top 50 companies
print("=" * 100)
print("TOP 50 COMPANIES - WITH SECTOR, COUNTRY, GROUP, AND REPORT COUNT")
print("=" * 100)
print(company_summary.head(50).to_string(index=False))

# Optional: Save to CSV
company_summary.to_csv('company_summary.csv', index=False)
print("\n✅ Full company summary saved to 'company_summary.csv'")

TOP 50 COMPANIES - WITH SECTOR, COUNTRY, GROUP, AND REPORT COUNT
                         Name                          Sector                                              Country      Group  report_count
                       TERUMO             Healthcare Products                                                Japan   Consumer             9
              Infosys Limited             Commercial Services                                                India  Financial             8
                   Iino Lines                       Logistics                                                Japan  Financial             7
                      Iwatani                Energy Utilities                                                Japan Industrial             7
                     NSK Ltd.                       Equipment                                                Japan Industrial             7
            Jtekt Corporation                 Metals Products                                  

In [29]:
# Count reports per company
company_freq = analysis_data['Name'].value_counts()

print("REPORTS PER COMPANY DISTRIBUTION")
print("=" * 50)
print(f"Companies with 1 report:  {(company_freq == 1).sum()}")
print(f"Companies with 2 reports: {(company_freq == 2).sum()}")
print(f"Companies with 3 reports: {(company_freq == 3).sum()}")
print(f"Companies with 4+ reports: {(company_freq >= 4).sum()}")

REPORTS PER COMPANY DISTRIBUTION
Companies with 1 report:  693
Companies with 2 reports: 208
Companies with 3 reports: 68
Companies with 4+ reports: 70


# next steps

- read in the text files

In [46]:
def load_text(folder, filename):
    path = os.path.join(folder, filename)
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()
    except:
        return None

In [47]:
final_data.columns

Index(['filename', 'company_name', 'year', 'Name', 'Year', 'file',
       'Organization_type', 'Size', 'Sector', 'Sec_SASB', 'Country', 'Region',
       'OECD', 'english_non_english', 'file_full_name', 'file_empty', 'Group',
       'text', 'text_length'],
      dtype='object')

In [48]:
folder = "./txt_converted"

final_data["text"] = final_data["filename"].apply(lambda x: load_text(folder, x))

In [49]:
final_data

,filename,company_name,year,Name,Year,file,Organization_type,Size,Sector,Sec_SASB,Country,Region,OECD,english_non_english,file_full_name,file_empty,Group,text,text_length
0,Camposol_2015.txt,Camposol,2015,Camposol,2015,Camposol_2015,Private company,MNE,Agriculture,Food and Beverage,Peru,Latin America & the Caribbean,No,non-english,Camposol_2015.txt,False,Other,\n 2014\n informe de\n\nsostenib...,234799
1,Anasoft_2015.txt,Anasoft,2015,Anasoft,2015,Anasoft_2015,Private company,SME,Technology Hardware,Technology and Communications,Slovak Republic,Europe,NaN,english,Anasoft_2015.txt,False,Consumer,sustainable development\n cop report 20...,97718
2,YunnanCoal&EnergyCompany_2013.txt,YunnanCoal&EnergyCompany,2013,Yunnan Coal & Energy Company,2013,YunnanCoal&EnergyCompany_2013,State-owned company,Large,Mining,Extractives and Minerals Processing,Mainland China,Asia,No,non-english,YunnanCoal&EnergyCompany_2013.txt,False,Industrial,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,47
3,ChinaDevelopmentFinancialHoldingCorporation(CD...,ChinaDevelopmentFinancialHoldingCorporation(CD...,2013,China Development Financial Holding Corporatio...,2013,ChinaDevelopmentFinancialHoldingCorporation(CD...,Private company,MNE,Financial Services,Financials,Taiwan,Asia,No,non-english,ChinaDevelopmentFinancialHoldingCorporation(CD...,False,Financial,中華開發金融控股公司\n\n2012 年企業社會責任報告書\n\n ...,58498
4,AECOM_2016.txt,AECOM,2016,AECOM,2016,AECOM_2016,Private company,Large,Other,Other,United States of America,Northern America,No,english,AECOM_2016.txt,False,Other,sustainability\nreport 2015\n\n ...,109973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5785,TianjinSamsungTechwinOpto-ElectronicCompany_20...,TianjinSamsungTechwinOpto-ElectronicCompany,2017,Tianjin Samsung Techwin Opto-Electronic Company,2017,TianjinSamsungTechwinOpto-ElectronicCompany_2017,Cooperative,Large,Technology Hardware,Technology and Communications,Mainland China,Asia,NaN,non-english,TianjinSamsungTechwinOpto-ElectronicCompany_20...,False,Consumer,天津天药药业股份有限公司\n\n 2...,7641
5786,ControladoraComercialMexicana_2012.txt,ControladoraComercialMexicana,2012,Controladora Comercial Mexicana,2012,ControladoraComercialMexicana_2012,Private company,Large,Commercial Services,Financials,Mexico,Latin America & the Caribbean,No,non-english,ControladoraComercialMexicana_2012.txt,False,Financial,controladora comercial mexicana ...,119810
5787,SinomachAutomobile_2014.txt,SinomachAutomobile,2014,Sinomach Automobile,2014,SinomachAutomobile_2014,State-owned company,Large,Automotive,Transportation,Mainland China,Asia,No,non-english,SinomachAutomobile_2014.txt,False,Industrial,\n编制说明 目录...,80120
5788,SNPTC_2016.txt,SNPTC,2016,SNPTC,2016,SNPTC_2016,State-owned company,Large,Energy,Extractives and Minerals Processing,Mainland China,Asia,NaN,non-english,SNPTC_2016.txt,False,Industrial,իે፞ԧ࡙ઑշ\n ᑂ䶳ᵥ⩢ࣾᆂ๶⡛㐬㞟㘪⎽\n\n报告规范\n\n 时间范围 ...,100788


In [50]:
# check for missing text
final_data["text"].isna().sum()

# check for empty files
final_data["text_length"] = final_data["text"].str.len()
final_data["text_length"].describe()

# they're all good 

count    5.682000e+03
mean     1.800863e+05
std      2.503855e+05
min      1.000000e+00
25%      3.019100e+04
50%      1.022780e+05
75%      2.254215e+05
max      4.552920e+06
Name: text_length, dtype: float64

In [51]:
## for now keep only english
df_en = final_data[final_data["english_non_english"] == "english"].copy()
df_en.shape
df_en

,filename,company_name,year,Name,Year,file,Organization_type,Size,Sector,Sec_SASB,Country,Region,OECD,english_non_english,file_full_name,file_empty,Group,text,text_length
1,Anasoft_2015.txt,Anasoft,2015,Anasoft,2015,Anasoft_2015,Private company,SME,Technology Hardware,Technology and Communications,Slovak Republic,Europe,NaN,english,Anasoft_2015.txt,False,Consumer,sustainable development\n cop report 20...,97718
4,AECOM_2016.txt,AECOM,2016,AECOM,2016,AECOM_2016,Private company,Large,Other,Other,United States of America,Northern America,No,english,AECOM_2016.txt,False,Other,sustainability\nreport 2015\n\n ...,109973
10,KimballOffice_2016.txt,KimballOffice,2016,Kimball Office,2016,KimballOffice_2016,Private company,Large,Consumer Durables,Consumer Goods,United States of America,Northern America,NaN,english,KimballOffice_2016.txt,False,Other,2017\n\n corporate\...,34932
12,CascadeEngineering_2016.txt,CascadeEngineering,2016,Cascade Engineering,2016,CascadeEngineering_2016,Private company,MNE,Other,Other,United States of America,Northern America,NaN,english,CascadeEngineering_2016.txt,False,Other,\ncascade engineering is comprised of nine bus...,52213
15,VodafoneQatar_2013.txt,VodafoneQatar,2013,Vodafone Qatar,2013,VodafoneQatar_2013,Private company,Large,Telecommunications,Technology and Communications,Qatar,Asia,No,english,VodafoneQatar_2013.txt,False,Consumer,vodafone qatar\ncsr report 2012-2013\n\na...,23526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5761,ADFIAP_2009.txt,ADFIAP,2009,ADFIAP,2009,ADFIAP_2009,NaN,SME,Non-Profit / Services,Non-Profit / Services,Philippines,Asia,NaN,english,ADFIAP_2009.txt,False,Other,...,126696
5767,SCJohnson_2014.txt,SCJohnson,2014,SC Johnson,2014,SCJohnson_2014,Private company,MNE,Household and Personal Products,Consumer Goods,United States of America,Northern America,NaN,english,SCJohnson_2014.txt,False,Consumer,the choices we make\nsc johnson 2014 public su...,160212
5768,OrientOverseasInternational_2016.txt,OrientOverseasInternational,2016,Orient Overseas International,2016,OrientOverseasInternational_2016,Private company,MNE,Logistics,Transportation,Hong Kong,Asia,No,english,OrientOverseasInternational_2016.txt,False,Financial,going green: we take it personally for sustain...,272750
5771,AhlstromCorporation_2011.txt,AhlstromCorporation,2011,Ahlstrom Corporation,2011,AhlstromCorporation_2011,Private company,Large,Forest and Paper Products,Renewable Resources & Alternative Energy,Finland,Europe,No,english,AhlstromCorporation_2011.txt,False,Other,sustainability report 2010\n\n\n\n\n ...,133454


In [52]:
# a cleaning function
import re

def clean_text_basic(text):
    if text is None:
        return None
    
    # lowercase
    text = text.lower()
    
    # remove line breaks
    text = text.replace("\n", " ")
    
    # remove digits
    text = re.sub(r"\d+", " ", text)
    
    # remove punctuation
    text = re.sub(r"[^\w\s]", " ", text)
    
    # remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [59]:
# apply the cleaning
df_en["text_clean"] = df_en["text"].apply(clean_text_basic)

In [60]:
# check that cleaning worked

df_en[["text","text_clean"]].iloc[0]

text          sustainable development\n        cop report 20...
text_clean    sustainable development cop report dear reader...
Name: 1, dtype: object

In [61]:
df_en["clean_len"] = df_en["text_clean"].str.len()
df_en["clean_len"].describe()

count    1.653000e+03
mean     1.556017e+05
std      1.635305e+05
min      5.800000e+01
25%      3.671800e+04
50%      1.094050e+05
75%      2.110850e+05
max      1.268557e+06
Name: clean_len, dtype: float64

In [62]:
df_en

,filename,company_name,year,Name,Year,file,Organization_type,Size,Sector,Sec_SASB,...,Region,OECD,english_non_english,file_full_name,file_empty,Group,text,text_length,text_clean,clean_len
1,Anasoft_2015.txt,Anasoft,2015,Anasoft,2015,Anasoft_2015,Private company,SME,Technology Hardware,Technology and Communications,...,Europe,NaN,english,Anasoft_2015.txt,False,Consumer,sustainable development\n cop report 20...,97718,sustainable development cop report dear reader...,42754
4,AECOM_2016.txt,AECOM,2016,AECOM,2016,AECOM_2016,Private company,Large,Other,Other,...,Northern America,No,english,AECOM_2016.txt,False,Other,sustainability\nreport 2015\n\n ...,109973,sustainability report delivering impactful sol...,42960
10,KimballOffice_2016.txt,KimballOffice,2016,Kimball Office,2016,KimballOffice_2016,Private company,Large,Consumer Durables,Consumer Goods,...,Northern America,NaN,english,KimballOffice_2016.txt,False,Other,2017\n\n corporate\...,34932,corporate sustainability report kimball has be...,15577
12,CascadeEngineering_2016.txt,CascadeEngineering,2016,Cascade Engineering,2016,CascadeEngineering_2016,Private company,MNE,Other,Other,...,Northern America,NaN,english,CascadeEngineering_2016.txt,False,Other,\ncascade engineering is comprised of nine bus...,52213,cascade engineering is comprised of nine busin...,36560
15,VodafoneQatar_2013.txt,VodafoneQatar,2013,Vodafone Qatar,2013,VodafoneQatar_2013,Private company,Large,Telecommunications,Technology and Communications,...,Asia,No,english,VodafoneQatar_2013.txt,False,Consumer,vodafone qatar\ncsr report 2012-2013\n\na...,23526,vodafone qatar csr report about vodafone bette...,14688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5761,ADFIAP_2009.txt,ADFIAP,2009,ADFIAP,2009,ADFIAP_2009,NaN,SME,Non-Profit / Services,Non-Profit / Services,...,Asia,NaN,english,ADFIAP_2009.txt,False,Other,...,126696,integrated annual and sustainability report co...,54134
5767,SCJohnson_2014.txt,SCJohnson,2014,SC Johnson,2014,SCJohnson_2014,Private company,MNE,Household and Personal Products,Consumer Goods,...,Northern America,NaN,english,SCJohnson_2014.txt,False,Consumer,the choices we make\nsc johnson 2014 public su...,160212,the choices we make sc johnson public sustaina...,76402
5768,OrientOverseasInternational_2016.txt,OrientOverseasInternational,2016,Orient Overseas International,2016,OrientOverseasInternational_2016,Private company,MNE,Logistics,Transportation,...,Asia,No,english,OrientOverseasInternational_2016.txt,False,Financial,going green: we take it personally for sustain...,272750,going green we take it personally for sustaina...,167059
5771,AhlstromCorporation_2011.txt,AhlstromCorporation,2011,Ahlstrom Corporation,2011,AhlstromCorporation_2011,Private company,Large,Forest and Paper Products,Renewable Resources & Alternative Energy,...,Europe,No,english,AhlstromCorporation_2011.txt,False,Other,sustainability report 2010\n\n\n\n\n ...,133454,sustainability report ahlstrom is a high perfo...,57564


### Here's the plan: 
### RQ1  What sustainability topics do Financial, Industrial, and Consumer sectors discuss in their disclosures?

Topic discovery by sector. 
> BERTopic (for now, or LDA)
> as a measure for topic prevalence by sector, we compute topic proportions per document and aggregate by sector 


clean topic text <br>
embeddings (document-level) <br>
BERTopic  <br>
topic per report  <br>
topic prevalence per sector  <br>
interpretation  <br>


# do not run the cells below they take forever :D

In [63]:
def lemmatize(text):
    doc = nlp(text[:1000000])
    return " ".join([t.lemma_ for t in doc if not t.is_punct and not t.is_space])

df_en["text_lemma"] = df_en["text"].apply(lemmatize)

KeyboardInterrupt: 

### RQ2  Did sustainability language shift from Brundtland keywords toward sector-specific keywords after the introduction of GRI G4 (2013), the EU NFRD (2014), and the UN SDGs (2015)?
 Measuring dictionary based longitudinal change

 > build a Brundtland dictionary 
 > build sector dictionaries 
 > count occurences in text_lemma 
 > compute yearly normalized frequencies

Sentence segmentation with spacy nlp
why? > measure sustainability topics at sentence-level, 

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=["ner","parser","tagger"])
nlp.add_pipe("sentencizer")

def split_sentences(text):
    doc = nlp(text[:1000000])  # safety cap
    return [s.text for s in doc.sents]

df_en["sentences"] = df_en["text"].apply(split_sentences)